## Face & Eye Detection using HAAR Cascade Classifiers

### Haar Cascade:

An object detection method that inputs Haar features into a series of classifiers (cascade) to identify objects in an image. They are trained to and very good at identify one type of object, however, in order to detect several at the same instance, we can use several of them in parallel (e.g. detecting eyes and faces together)

HAAR Classifier are trained using lots of **positive images** (i.e. images with the object present) and **negative images** (i.e. images without the object present).

We then extract features using **sliding windows of rectangular blocks**. These features are single valued and are calculated by subtracting the sum of pixel intensities under the white rectangles from the black rectangles. However this is a ridiculous number of calculations, even for a base window of 24 x 24 pixels (180,000 features generated). So the researchers devised a method called **Integral Images** that computed this with four array references.

However, they still had 180,000 features and the majority of them added no real value because those areas didn't have a face.

Boosting was then used to determine the most informative features, with Freaund & Schapire's AdaBoost the algorithm of choice due to its ease of implementation. Bosting is the process by which we use weak classifiers to build strong classfiers, simply by assigning heavier weghted penalties on incorrect classifications. Reducing the 180,000 features to 6000, which is still quite a bit features.

Those 6000 featres, some will be more informative han others. What if we used the most informative features to first check whether the regions can potentially have a face (false positives will be no big deal). Doing so eliminates the need for calculating all 6000 features at once.

--> this concept is called the Cascade of classifiers - for face detection, the Viola Jones method used 38 stages.

#### Cascade Classifiers Flow

Load Classifier --> Pass image to Classifier/Detector --> Get location/ROI for detected objects --> Draw Rectangle over Detected Objects

In [2]:
import numpy as np
import cv2

In [4]:
# We point OpenCV's CascadeClassifier function to where our classifier (XML file format) is stored
# This creates the face classifier object here
face_classifier = cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')

# Load our image then convert it to grayscale
image = cv2.imread('images/Trump.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Our classifier returns the ROI of the detected face as a tuple
# It stores the top left coordinate and the bottom right coordiantes
# This actually gives us the region of interests
# `faces` is location of many faces instead of just one face
faces = face_classifier.detectMultiScale(gray, 1.3, 5)

# When no faces detected, face_classifier returns and empty tuple
if faces is ():
    print("No faces found")

# We iterate through our faces array and draw a rectangle
# over each face in faces
for (x, y, w, h) in faces:
    cv2.rectangle(image, (x, y), (x+w, y+h), (127, 0, 255), 2)
    cv2.imshow('Face Detection', image)
    cv2.waitKey(0)
    
cv2.imwrite('images/Trump_detection.jpg', image)
cv2.destroyAllWindows()

Original Image | Face Detection
- | -
![alt](images/Trump.jpg) | ![alt](images/Trump_detection.jpg)

### Let's combine face and eye detection

In [ ]:
import numpy as np
import cv2

In [5]:
# Difference here comparing with above is that we are having two classifier now
face_classifier = cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')
eye_classifier = cv2.CascadeClassifier('Haarcascades/haarcascade_eye.xml')
 
img = cv2.imread('images/Trump.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = face_classifier.detectMultiScale(gray, 1.3, 5)

# When no faces detected, face_classifier returns and empty tuple
if faces is ():
    print("No Face Found")

for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(127,0,255),2)
    cv2.imshow('img',img)
    cv2.waitKey(0)
    # once face is found, we crop that image
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    eyes = eye_classifier.detectMultiScale(roi_gray)
    # and feed the cropped image to the eye classifier
    count = 0
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(255,255,0),2)
        cv2.imshow('img',img)
        cv2.waitKey(0)
        count += 1
        cv2.imwrite('images/Trump_eyeDetection{num}.jpg'.format(num=count), img)
    
cv2.destroyAllWindows()

Original Image | Face Detection | Eye Detection1 | Eye Detection2
- | - | - | -  
![alt](images/Trump.jpg) | ![alt](images/Trump_detection.jpg) | ![alt](images/Trump_eyeDetection2.jpg) | ![alt](images/Trump_eyeDetection3.jpg)

### Let's make a live face & eye detection, keeping the face inview at all times

In [2]:
import cv2
import numpy as np

In [3]:
face_classifier = cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')
eye_classifier = cv2.CascadeClassifier('Haarcascades/haarcascade_eye.xml')

def face_detector(img, size=0.5):
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img
    
    for (x,y,w,h) in faces:
        x = x - 50
        w = w + 50
        y = y - 50
        h = h + 50
        # instead of drawing a rectangle for the face although it is being done here we actually just crop the rectangle out 
        # and once we have cropped that image of all of this,we detec eyes in that image. 
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        eyes = eye_classifier.detectMultiScale(roi_gray)
        # eye detection
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,0,255),2) 
            
    roi_color = cv2.flip(roi_color,1)
    return roi_color

cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    cv2.imshow('Our Face Extractor', face_detector(frame))
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      

### Tuning Cascade Classifiers

*ourClassifier*.**detectMultiScale**(input image, **Scale Factor** , **Min Neighbors**)

- **Scale Factor**
Specifies how much we reduce the image size each time we scale. E.g. in face detection we typically use 1.3. This means we reduce the image by 30% each time it’s scaled. Smaller values, like 1.05 will take longer to compute, but will increase the rate of detection.



- **Min Neighbors**
Specifies the number of neighbors each potential window should have in order to consider it a positive detection. Typically set between 3-6. 
It acts as sensitivity setting, low values will sometimes detect multiples faces over a single face. High values will ensure less false positives, but you may miss some faces.  
